In [1]:
import pandas as pd
import os
from docx import Document
import re

In [2]:
data_path = os.path.abspath('data')

In [3]:
excel_file = os.path.join(data_path, 'Ficha_Afericao.xlsm')

In [4]:
df = pd.read_excel(excel_file, usecols=range(4))

In [5]:
df

,Classificação,Documento,Nome,Cargo
0,1,XXXXXXXX-1,BRUNO LUIS SIMÕES GERALDO,Analista de Políticas Públicas
1,2,XXXXXXXX-2,JUSSARA MARIA DOS SANTOS,Analista de Políticas Públicas
2,3,XXXXXXXX-3,JOICE APARECIDA SILVA,Analista de Políticas Públicas
3,4,XXXXXXXX-4,THIAGO OLIVEIRA CHAVES,Analista de Políticas Públicas
4,5,XXXXXXXX-5,RICARDO JESUS,Analista de Políticas Públicas
5,6,XXXXXXXX-6,SILVA MEIRELES,Analista de Políticas Públicas
6,7,XXXXXXXX-7,ANA CRISTINA PRADO,Analista de Políticas Públicas
7,8,XXXXXXXX-8,MARCIA MORALEZ,Analista de Políticas Públicas
8,9,XXXXXXXX-9,REGINALDO FREITAS,Analista de Políticas Públicas
9,10,XXXXXXXX-10,ROSANA GARCIA,Analista de Políticas Públicas


In [6]:
df = df.sort_values(by='Nome ')

In [7]:
word_path = os.path.join(data_path, 'Ficha de aferição - Análise fotográfica.docx')


In [8]:
def get_paragrafos(doc):
    
    interesse = []
    for i, p in enumerate(doc.paragraphs):
        if p.text.startswith('Classificação'):
            dupla = (p, doc.paragraphs[i+1])
            interesse.append(dupla)
    
    return interesse

In [9]:
def colocar_nome(p, nome):
    
    pat = '<<nome do candidato \d>>'
    
    chave = re.findall(pat, p.text)[0]
    
    p.text = p.text.replace(chave, nome)
    
    return p.text

In [10]:
def colocar_doc(p, documento):
    
    pat = '<<documento \d>>'
    
    chave = re.findall(pat, p.text)[0]
    
    p.text = p.text.replace(chave, documento)
    
    return p.text

In [11]:
def gerar_grupos(df):

    linhas = []
    for i, row in df.iterrows():

        linhas.append((row['Documento'], row['Nome ']))
    grupos = []
    ini = 0
    for i in range(len(linhas)):

        if i != 0 and (i%4)==0:
            grupos.append(linhas[ini:i])
            ini = i

    grupos.append(linhas[ini:])
    
    return grupos

In [12]:
grupos = gerar_grupos(df)

In [13]:
padrao_nome_doc = 'formulario_{num}.docx'

for i, grupo in enumerate(grupos):
    
    doc = Document(word_path)
    paragrafos = get_paragrafos(doc)
    nome_doc = padrao_nome_doc.format(num=i)
    pareado = list(zip(grupo, paragrafos))
    
    for pessoa, linhas in pareado:
        colocar_doc(linhas[0], pessoa[0])
        colocar_nome(linhas[1], pessoa[1])
        doc.save(nome_doc)
